In [1]:
from scrap import Scraper
from RAG_QnA import RAG_Model

In [10]:
url = "https://www.nu.edu.pk/Admissions/HowToApply"
scrp = Scraper(url)

In [11]:
print(scrp.scrape_website())

The procedure for admission is as follows: Information on various Admission Tests is available on the University’s admission web-site
                             
 admissions.nu.edu.pk The online admission test is taken on a computer. The candidate should be familiar with the use of mouse and the keyboard. Each candidate is given a different examination. The sections of a test (English, Analytical skills, Math etc) appear in a random sequence. Moreover, questions related to different topics also appear before the candidate in a random sequence. In each section, a candidate can move forward and backwards to review the responses selected. The candidate can change his or her response to any question within the current section. Each section of the admission test has a fixed duration. Each section is timed independently and time saved in one section is not carried forward to the next sections. At the end of the test, a summary of the results is displayed on screen. The duration of the Onli

In [6]:
rag_model = RAG_Model()

2024-07-30 10:49:26.495117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 10:49:26.519970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 10:49:26.527135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 10:49:28.265791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [12]:
rag_model.load_Database()

In [14]:
print(rag_model.generateResponse(prompt="How are to apply for univeristy?"))

1. Visit the official website [admissions.nu.edu.pk](http://www.admissions.nu.edu.pk) or [www.nu.edu.pk] to access the application portal.
        2. Fill out the online application form with personal information, educational background, and work experience.
        3. Pay the admission fee of Rs 30,000 (one-time) through the available payment methods.
        4. Submit supporting documents like No Objection Certificate (for foreign nationals), transcripts, and test scores such as SAT for undergraduates and GRE for graduates.
        5. Upload the completed application and required documents, then click submit.
        6. You will receive an acknowledgement email after successful submission, followed by further instructions.
        7. Check your email regularly for updates regarding your application status.</s>


In [13]:
print(rag_model.window_mem.buffer_as_str)

Human: Tell me Fee structures for masters in computer science?
AI: 1. Admission Fee: Rs 30,000 (one-time), waived for graduates of the same university.
        2. Refundable Security Deposit: Rs 20,000.
        3. Tuition Fee per credit hour: Not specified in the provided context, revised annually.
        4. Student Activities Fund: Rs 2,500 per semester.
        5. Miscellaneous fees include Additional Transcript Fee (Rs. 300), Campus Transfer Fee (Rs. 5,000), Temporary Campus Transfer Fee (Rs. 2,000), Degree Correction Fee (Rs. 2,000), Semester Freeze Fee (Rs. 2,000 if registered within two weeks of classes start), Late Registration Fee/Freeze Fee/Course Withdrawal (not specified).
        6. Scholarships are available for top performers and the HEC, Government of Punjab, and other organizations offer financial assistance for MS & PhD degrees.</s>
